# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 15 different words. 

In [20]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator, TabularDataset
import pandas as pd
import re

#torch.backends.cudnn.enabled = False
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
hardware = 'cuda:1' if torch.cuda.is_available() else 'cpu'
#hardware = 'cpu'
device = torch.device(hardware)

batch_size = 64
learning_rate = 0.001
epochs = 3
embedding_size = 512
embedding_dim = 512


# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 15 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [21]:
def data_split(path_to_dataset):
    with open(path_to_dataset) as f:
        lines = f.read().splitlines()
        lines_tokenized = [re.split(r'\t+', l) for l in lines]

        df = pd.DataFrame(lines_tokenized, columns=['word-sense', 'word-form', 'index_of_word', 'context'])
        df_test_data = df.sample(frac=0.20)
        df_training_data = df.drop(df_test_data.index)

        df_training_data.to_csv('training_data.csv', index=False, sep='\t')
        df_test_data.to_csv('testing_data.csv', index=False, sep='\t')

#data_split('wsd-data/wsd_data_subset.txt')
data_split('wsd-data/wsd_data.txt')

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [22]:
def mcs_baseline(data):
    sense_freq = data.groupby(['word-form', 'word-sense'], as_index=False).size()
    word_freq = data.groupby('word-form', as_index=False).size()
    mcs = sense_freq.loc[sense_freq.groupby('word-form')['size'].idxmax()].reset_index()
    mcs['accuracy'] = (mcs['size'] / word_freq['size'])
    print("MCS accuracy mean: {0:.0%}".format(mcs['accuracy'].mean()))

    mcs_output = mcs.to_string(columns=['word-form', 'accuracy'], formatters={'accuracy': '{:,.2%}'.format})
    print("MCS accuracy table:")
    print(mcs_output)

In [23]:
df = pd.read_csv('training_data.csv', sep='\t')
mcs_baseline(df)

MCS accuracy mean: 28%
MCS accuracy table:
         word-form accuracy
0         active.a   32.73%
1            bad.a   61.61%
2          bring.v   20.52%
3          build.v   21.39%
4           case.n   20.14%
5         common.a   26.22%
6       critical.a   28.01%
7         extend.v   17.66%
8           find.v   23.31%
9         follow.v   14.25%
10         force.n   16.43%
11          hold.v   14.75%
12          keep.v   39.32%
13          lead.v   18.03%
14          life.n   22.98%
15          line.n   84.64%
16         major.a   30.67%
17      national.a   20.29%
18         order.n   21.76%
19      physical.a   23.98%
20         place.n   25.51%
21         point.n   34.51%
22      position.n   20.06%
23      positive.a   35.36%
24  professional.a   22.00%
25       regular.a   22.85%
26      security.n   20.86%
27           see.v   62.47%
28         serve.v   15.27%
29          time.n   27.86%


### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [24]:
def dataloader(path):
    whitespacer = lambda x: x.split(' ')

    LABEL = Field(tokenize = whitespacer, batch_first = True)
    TOKEN = Field(tokenize = whitespacer, batch_first = True, lower = True,)

    train, test = TabularDataset.splits(
        path   = path,
        train  = 'training_data.csv',
        test   = 'testing_data.csv',
        format = 'csv',
        fields = [
            ('word_sense', LABEL),
            ('word_form', TOKEN),
            ('index_of_word', TOKEN),
            ('context', TOKEN)],
        csv_reader_params = {'delimiter':'\t', 'quotechar':'½'})

    LABEL.build_vocab(train)
    TOKEN.build_vocab(train)

    train_iter, test_iter = BucketIterator.splits(
        (train, test),
        batch_size        = batch_size,
        sort_within_batch = True,
        sort_key          = lambda x: len(x.word_form),
        shuffle           = True,
        device            = device)

    return train_iter, test_iter, TOKEN.vocab, LABEL.vocab

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [25]:
class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_size, label_count, embedding_dim):
        super(WSDModel_approach1, self).__init__()
        # 1) An embedding layer: a dictionary from which we can obtain word 
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 2) A LSTM-module to obtain contextual representations
        self.lstm = nn.LSTM(embedding_dim, embedding_dim)
        # 3) A classifier that compute scores for each word-sense given *some* input     
        self.fc = nn.Linear(embedding_dim, label_count)
        #self.dropout = nn.Dropout(0.2)
    
    def forward(self, batch):
        # 1) For each word in the sentence, obtain word embeddings
        embedded_context = self.embeddings(batch.context)
        # 2) Run the embedded sentences through the RNN
        #output, (hs, cs) = self.lstm(embedded_context)
        output, (_, _) = self.lstm(embedded_context)
        #ouput = self.dropout(output)
        #print("output2", output.shape)
        output = torch.sum(output, dim = 1)
        output = self.fc(output)
        # 3) Select the appropriate hidden state
        # 4) Predict the word-sense based on:
        #       The index of the target word
        #target_index = batch.index_of_word

        return output
    
    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [26]:
class WSDModel_approach2(nn.Module):
    def __init__(self, word_count, embedding_dim):
        self.embeddings = nn.Embedding(word_count, embedding_dim)   # 1) An embedding layer: a dictionary from which we can obtain word embeddings
        self.rnn = ...          # 2) A LSTM-module to obtain contextual representations
        self.classifier = ...   # 3) A classifier that compute scores for each word-sense given *some* input
    
    def forward(self):
        # 1) For each word in the sentence, obtain word embeddings
        # 2) Run the embedded sentences through the RNN
        # 3) Select the appropriate hidden state
        # 4) Predict the word-sense based on:
        #       The final hidden state given by the RNN
        
        return 0 #predictions

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [27]:
path_to_folder = "./"
train_iter, test_iter, vocab, labels = dataloader(path_to_folder)

In [31]:
vocab_size = len(vocab)
label_count = len(labels)
model = WSDModel_approach1(vocab_size, label_count, embedding_dim)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(epochs):
    print("Epoch", epoch)
    total_loss = 0
    for i, batch in enumerate(train_iter):
        expected = batch.word_sense
        predicted = model(batch)
        loss = loss_function(predicted.view(-1, label_count), expected.view(-1))
        total_loss += loss.item()
        print(f'\rAvg loss: {total_loss/(i+1)}', end='') 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print()

# test model after all epochs are completed

Epoch 0
Avg loss: 5.441224172413663
Epoch 1
Avg loss: 1.6745914710934355
Epoch 2
Avg loss: 1.1131305292828477


# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [29]:
def dataloader_for_bert(path_to_file, batch_size):
    ...

In [30]:
class BERT_WSD(nn.Module):
    def __init__(self, ...):
        # your code goes here
        self.bert = ...
        self.classifier = ...
    
    def forward(self, batch):
        # your code goes here
        
        return predictions

SyntaxError: invalid syntax (<ipython-input-30-45680f45608d>, line 2)

In [ ]:
loss_function = ...
optimizer = ...
model = ...

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf